- wrap_function表示传入的函数lsp_func()一个别名一样，共享lsp_func()的传入参数，这样的话，我们可以把中间结果加入到auth这个参数中，继续传给lsp_func()

- 之后，我们修改了传入参数auth，继续给function(*args, **auth)。

- @wraps(function)：作用就是保留function函数的元信息，比如function.__name__, function.__doc__




## decorator表示包装传入的函数function，这是固定写法，但是如果没有check_permission，这样的装饰器就不能传递参数，因此，在外边包装个check_permission
## 函数，可以给装饰器也传递参数 

## return wrap_function固定写法，返回被修饰的函数

In [14]:

def check_permission(user_roles):
    def decorator(function):
        @wraps(function)
        def wrap_function(*args, **auth):
            print(f'auth 0 = {auth}')
            auth['y'] = 10
            return function(*args, **auth)
        return wrap_function
    return decorator


from functools import wraps

@check_permission(['a', 'b'])
def lsp_func(**auth):
    print(f'auth 2 = {auth}')
    return auth

res = lsp_func(x=9)
res

auth 0 = {'x': 9}
auth 2 = {'x': 9, 'y': 10}


{'x': 9, 'y': 10}

In [ ]:
## 去掉@wraps(function)

In [8]:
lsp_func.__name__

'wrap_function'

In [ ]:
## 加上@wraps(function)

In [10]:
lsp_func.__name__

'lsp_func'

## 以下就是没有参数的装饰器写法

In [23]:
import time
from functools import wraps
def timethis(func): 
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(args)
        start = time.process_time()
        r = func(*args, **kwargs)
        end = time.process_time()
        print('{}.{} : {}'.format(func.__module__, func.__name__, end - start)) 
        return r
    return wrapper

@timethis
def func_test(n:int=3):
    while n > 0:
        n -= 1
#         if n % 1000 == 0:
#             print(f'n = {n}')
    
func_test(10000000)

(10000000,)
__main__.func_test : 0.6585159999999988


In [24]:
func_test.__annotations__

{'n': int}

## wrapper是被装饰的函数

In [ ]:
def add_config(name: str, dtype: type, default=None, desc: str = ''):
    """
    作为修饰器，给修饰的可调用过程（方法，类）加上一个新的配置项。并在该过程调用时注入第一个参数为config。
    :param name: 配置名称
    :param dtype: 配置的数据类型，仅支持 int, str, float。如果想使用bool，请用int的0和1代替。
    :param default: 默认项，建议设置。如果想表示None，可以用空字符串之类表示。
    :param desc: 描述。help中显示的内容。
    """

    def wrapper(cls):
        cls = _insert_config(cls)  # [train, evaluate, predict, server]
        group_name = _get_hack_name(cls)  # [train, evaluate, predict, server]
        global_config = GlobalConfig()
        global_config.get_config_group(group_name).add_config(name, dtype, default, desc)
        print(f'添加参数name={name}')
        return cls

    return wrapper
